# Concrete Compressive Strength Prediction C

## 1. Import important libraries and packages

In [1]:
import pandas as pd
import numpy as np
import keras
from keras.models import Sequential
from keras.layers import Dense
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split

Using TensorFlow backend.


### Import data

In [2]:
concrete_data = pd.read_csv('concrete_data.csv')
concrete_data.head()

,Cement,Blast Furnace Slag,Fly Ash,Water,Superplasticizer,Coarse Aggregate,Fine Aggregate,Age,Strength
0,540.0,0.0,0.0,162.0,2.5,1040.0,676.0,28,79.99
1,540.0,0.0,0.0,162.0,2.5,1055.0,676.0,28,61.89
2,332.5,142.5,0.0,228.0,0.0,932.0,594.0,270,40.27
3,332.5,142.5,0.0,228.0,0.0,932.0,594.0,365,41.05
4,198.6,132.4,0.0,192.0,0.0,978.4,825.5,360,44.30


## Evaluate the imported data

In [3]:
concrete_data.shape 

(1030, 9)

In [4]:
concrete_data.describe()

,Cement,Blast Furnace Slag,Fly Ash,Water,Superplasticizer,Coarse Aggregate,Fine Aggregate,Age,Strength
count,1030.000000,1030.000000,1030.000000,1030.000000,1030.000000,1030.000000,1030.000000,1030.000000,1030.000000
mean,281.167864,73.895825,54.188350,181.567282,6.204660,972.918932,773.580485,45.662136,35.817961
std,104.506364,86.279342,63.997004,21.354219,5.973841,77.753954,80.175980,63.169912,16.705742
min,102.000000,0.000000,0.000000,121.800000,0.000000,801.000000,594.000000,1.000000,2.330000
25%,192.375000,0.000000,0.000000,164.900000,0.000000,932.000000,730.950000,7.000000,23.710000
50%,272.900000,22.000000,0.000000,185.000000,6.400000,968.000000,779.500000,28.000000,34.445000
75%,350.000000,142.950000,118.300000,192.000000,10.200000,1029.400000,824.000000,56.000000,46.135000
max,540.000000,359.400000,200.100000,247.000000,32.200000,1145.000000,992.600000,365.000000,82.600000


In [5]:
concrete_data.isnull().sum()

Cement                0
Blast Furnace Slag    0
Fly Ash               0
Water                 0
Superplasticizer      0
Coarse Aggregate      0
Fine Aggregate        0
Age                   0
Strength              0
dtype: int64

In [6]:
concrete_data_columns = concrete_data.columns
predictors = concrete_data[concrete_data_columns[concrete_data_columns != 'Strength']] #All the columns except strength
predictors_columns = predictors.columns
predictors = predictors[predictors_columns[predictors_columns != 'Age']] #All the columns except age
target = concrete_data['Strength'] 

In [7]:
predictors.head()

,Cement,Blast Furnace Slag,Fly Ash,Water,Superplasticizer,Coarse Aggregate,Fine Aggregate
0,540.0,0.0,0.0,162.0,2.5,1040.0,676.0
1,540.0,0.0,0.0,162.0,2.5,1055.0,676.0
2,332.5,142.5,0.0,228.0,0.0,932.0,594.0
3,332.5,142.5,0.0,228.0,0.0,932.0,594.0
4,198.6,132.4,0.0,192.0,0.0,978.4,825.5


In [8]:
target.head()

0    79.99
1    61.89
2    40.27
3    41.05
4    44.30
Name: Strength, dtype: float64

# B. NORMALIZE THE DATA

In [9]:
mean_predictors = np.mean(predictors) #Find the mean of the predictors
std_predictors = np.std(predictors) #Find the standard deviation of the predictors
predictors = (predictors - mean_predictors)/std_predictors
mean_target = np.mean(target) #Find the mean of the target
std_target = np.std(target) #Find the standard deviation of the target
target = (target - mean_target)/std_target

# A. Build a baseline model

## 1. Randomly split the data into a training and test sets by holding 30% of the data for testing. You can use the train_test_split helper function from Scikit-learn.

In [10]:
X_train,X_test,y_train,y_test = train_test_split(predictors,target,test_size = 0.3,random_state=85)

In [11]:
column_nums = X_train.shape[1]

Use the Keras library to build a neural network with the following:

- One hidden layer of 10 nodes, and a ReLU activation function

- Use the adam optimizer and the mean squared error as the loss function.


In [12]:
def model_A():
    model = Sequential()
    model.add(Dense(10,activation='relu',input_shape = (column_nums,)))
    model.add(Dense(1))
    model.compile(optimizer='adam',loss='mean_squared_error')
    return model

# C. Increate the number of epochs
## Repeat Part B but use 100 epochs this time for training.

In [13]:
model = model_A()
model.fit(X_train,y_train,epochs=100,verbose=0) #Made the logg of the model development silent verbose = 0

## 3. Evaluate the model on the test data and compute the mean squared error between the predicted concrete strength and the actual concrete strength. You can use the mean_squared_error function from Scikit-learn.

In [14]:
y_pred = model.predict(X_test)
mse = mean_squared_error(y_pred,y_test)
print("The Mean Squared Error between the predicted and actual strength is "+str(mse))

The Mean Squared Error between the predicted and actual strength is 0.5046514181625031


## 4. Repeat steps 1 - 3, 50 times, i.e., create a list of 50 mean squared errors.

In [15]:
mselist = [] #created an empty list
for i in range (0,50):
    X_train,X_test,y_train,y_test = train_test_split(predictors,target,test_size = 0.3,random_state=85)
    #Used the function model_A() to prepare the model
    model = model_A()
    print("Started training: "+str(i+1))
    model.fit(X_train,y_train,epochs=100,verbose=0)
    y_pred = model.predict(X_test)
    mse = mean_squared_error(y_pred,y_test)
    print("Finished training: "+str(i+1)+"\n The MSE is: "+str(mse))
    mselist.append(mse)
print("Done")
print(mselist)

Started training: 1
Finished training: 1
 The MSE is: 0.5000285727991601
Started training: 2
Finished training: 2
 The MSE is: 0.5371238372016115
Started training: 3
Finished training: 3
 The MSE is: 0.5342870100253201
Started training: 4
Finished training: 4
 The MSE is: 0.5221929280625583
Started training: 5
Finished training: 5
 The MSE is: 0.5098829256152385
Started training: 6
Finished training: 6
 The MSE is: 0.5036394779232898
Started training: 7
Finished training: 7
 The MSE is: 0.514028124161202
Started training: 8
Finished training: 8
 The MSE is: 0.5398287854214069
Started training: 9
Finished training: 9
 The MSE is: 0.5031623391823977
Started training: 10
Finished training: 10
 The MSE is: 0.5289670884189571
Started training: 11
Finished training: 11
 The MSE is: 0.5074908758773738
Started training: 12
Finished training: 12
 The MSE is: 0.5264642937912578
Started training: 13
Finished training: 13
 The MSE is: 0.5235491205439923
Started training: 14
Finished training: 14
 

In [16]:
mean_of_mse = np.mean(mselist) #Compute the mean of the 50 mean squared errors
std_of_mse = np.std(mselist) #Compute the standard deviation of the 50 mean squared errors

In [17]:
print("The mean of the 50 mean squared errors is: "+str(mean_of_mse))
print("The standard deviation of the 50 mean squared errors is: "+str(std_of_mse))

The mean of the 50 mean squared errors is: 0.5201474419839167
The standard deviation of the 50 mean squared errors is: 0.01295356620819413


### How does the mean of the mean squared errors compare to that from Step B?
The mean of mean squared errors in <strong>C</strong> is relatively small compared to <strong>B</strong>.
<br/>During my first runtime the mean of the mean squared errors in <strong>B</strong> was arround <em>0.531</em> while mean of the mean squared errors in <strong>C</strong> was arround <em>0.5201</em>